In [29]:
%pip install pandas
%pip install openpyxl
%pip install scikit-learn


You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 8.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 8.8 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd 

df_demo = pd.read_csv("Coursework Data/DemoStats.csv")
df_household = pd.read_csv("Coursework Data/HouseholdSpend.csv")


In [2]:
import pandas as pd

# DemoStats metadata
demo_change_list = pd.read_csv("Documentation/DemoStats 2024 - Variables Change List.csv")
demo_vars_list = pd.read_csv("Documentation/DemoStats 2024 - Variables List.csv")

# HouseholdSpend metadata
household_change_list = pd.read_excel("Documentation/HouseholdSpend 2024 - Variables Change List.xlsx")
household_vars_list = pd.read_excel("Documentation/HouseholdSpend 2024 - Variables List.xlsx")


In [4]:
# (2) Define the terms we want to detect
search_terms = ["pension", "retirement", "insurance"]

# (3) A helper function to scan all relevant columns in a DataFrame
def find_keywords(df, df_name, columns_to_check, keywords):
    matches = []
    for idx, row in df.iterrows():
        for col in columns_to_check:
            val = str(row.get(col, "")).lower()
            for term in keywords:
                if term in val:
                    # Record a match
                    matches.append({
                        "DataFrame": df_name,
                        "RowIndex": idx,
                        "Column": col,
                        "Variable": row.get("Variable", ""),
                        "MatchedTerm": term,
                        "FullText": val
                    })
    return matches

# (4) Perform the searches
metadata_search_results = []

# DemoStats
metadata_search_results += find_keywords(
    demo_change_list, "DemoStats_Change_List",
    ["Variable", "Description", "Category"], search_terms
)
metadata_search_results += find_keywords(
    demo_vars_list, "DemoStats_List",
    ["Variable", "Description", "Hierarchy Description", "Category"], search_terms
)

# HouseholdSpend
metadata_search_results += find_keywords(
    household_change_list, "HouseholdSpend_Change_List",
    ["Variable", "Description", "Category"], search_terms
)
metadata_search_results += find_keywords(
    household_vars_list, "HouseholdSpend_List",
    ["Variable", "Description", "Category", "Consumption"], search_terms
)

# (5) Display or print results
if metadata_search_results:
    df_matches = pd.DataFrame(metadata_search_results)
    print("FOUND THE FOLLOWING MATCHES:")
    print(df_matches)
else:
    print("No references to 'pension', 'retirement', or 'insurance' found in these metadata files.")

FOUND THE FOLLOWING MATCHES:
              DataFrame  RowIndex                 Column    Variable  \
0        DemoStats_List       818            Description  ECYINDFINA   
1        DemoStats_List       818  Hierarchy Description  ECYINDFINA   
2   HouseholdSpend_List        22            Description    HSEP001S   
3   HouseholdSpend_List        22            Description    HSEP001S   
4   HouseholdSpend_List        22            Description    HSEP001S   
5   HouseholdSpend_List        37            Description     HSSH006   
6   HouseholdSpend_List        43            Description     HSSH015   
7   HouseholdSpend_List        47            Description     HSSH019   
8   HouseholdSpend_List        96            Description     HSSH044   
9   HouseholdSpend_List       170            Description     HSHC022   
10  HouseholdSpend_List       173            Description     HSHC025   
11  HouseholdSpend_List       224            Description     HSTR032   
12  HouseholdSpend_List       300  

In [5]:
df_keyword_matches = pd.DataFrame(metadata_search_results)

# 4) Build lists of pension/retirement/insurance variables from the results
demo_matches = df_keyword_matches[df_keyword_matches["DataFrame"].str.contains("DemoStats")]
household_matches = df_keyword_matches[df_keyword_matches["DataFrame"].str.contains("HouseholdSpend")]

demo_drop_vars = sorted(set(demo_matches["Variable"]))
household_drop_vars = sorted(set(household_matches["Variable"]))

print("DEMOStats pension/retirement/insurance variables:", demo_drop_vars)
print("HOUSEHOLDSpend pension/retirement/insurance variables:", household_drop_vars)

DEMOStats pension/retirement/insurance variables: ['ECYINDFINA']
HOUSEHOLDSpend pension/retirement/insurance variables: ['HSEP001', 'HSEP001S', 'HSEP002', 'HSEP003', 'HSEP004', 'HSEP005', 'HSEP006', 'HSEP007', 'HSEP008', 'HSEP009', 'HSHC022', 'HSHC025', 'HSRV011', 'HSSH006', 'HSSH015', 'HSSH019', 'HSSH044', 'HSTR032']


In [6]:
#Find “No Longer Available” variables in the Change Lists
demo_no_longer = demo_change_list[demo_change_list["Change Type"]
                                  .str.lower().str.contains("no longer available")]
household_no_longer = household_change_list[household_change_list["Change Type"]
                                            .str.lower().str.contains("no longer available")]

demo_no_longer_list = demo_no_longer["Variable"].unique().tolist()
household_no_longer_list = household_no_longer["Variable"].unique().tolist()

print("DEMOStats 'No Longer Available':", demo_no_longer_list)
print("HOUSEHOLDSpend 'No Longer Available':", household_no_longer_list)

# Combined lists for each data product
demo_all_drops = sorted(set(demo_drop_vars + demo_no_longer_list))
household_all_drops = sorted(set(household_drop_vars + household_no_longer_list))



DEMOStats 'No Longer Available': ['EHYMTN75P', 'EHYCFSC1C', 'EHYCFSC2C', 'EHYCFSC3C', 'EHYCFSM1C', 'EHYCFSM2C', 'EHYCFSM3C', 'EHYCFSCL1C', 'EHYCFSCL2C', 'EHYCFSCL3C', 'EHYCFSLP1C', 'EHYCFSLP2C', 'EHYCFSLP3C', 'EHYCFSFP1C', 'EHYCFSFP2C', 'EHYCFSFP3C', 'EHYCFSMP1C', 'EHYCFSMP2C', 'EHYCFSMP3C', 'EHYHFSC1C', 'EHYHFSC2C', 'EHYHFSC3C', 'EHYHFSM1C', 'EHYHFSM2C', 'EHYHFSM3C', 'EHYHFSCL1C', 'EHYHFSCL2C', 'EHYHFSCL3C', 'EHYHFSLP1C', 'EHYHFSLP2C', 'EHYHFSLP3C', 'EHYHFSFP1C', 'EHYHFSFP2C', 'EHYHFSFP3C', 'EHYHFSMP1C', 'EHYHFSMP2C', 'EHYHFSMP3C', 'ECYMTN75P', 'ECYCFSC1C', 'ECYCFSC2C', 'ECYCFSC3C', 'ECYCFSM1C', 'ECYCFSM2C', 'ECYCFSM3C', 'ECYCFSCL1C', 'ECYCFSCL2C', 'ECYCFSCL3C', 'ECYCFSLP1C', 'ECYCFSLP2C', 'ECYCFSLP3C', 'ECYCFSFP1C', 'ECYCFSFP2C', 'ECYCFSFP3C', 'ECYCFSMP1C', 'ECYCFSMP2C', 'ECYCFSMP3C', 'ECYHFSC1C', 'ECYHFSC2C', 'ECYHFSC3C', 'ECYHFSM1C', 'ECYHFSM2C', 'ECYHFSM3C', 'ECYHFSCL1C', 'ECYHFSCL2C', 'ECYHFSCL3C', 'ECYHFSLP1C', 'ECYHFSLP2C', 'ECYHFSLP3C', 'ECYHFSFP1C', 'ECYHFSFP2C', 'ECYHFSFP3C'

In [7]:
# -----------------------------
# 2) CONVERT THE MATCHES INTO DROP LISTS
# -----------------------------
df_matches_demo = df_matches[df_matches["DataFrame"].str.contains("DemoStats")]
df_matches_household = df_matches[df_matches["DataFrame"].str.contains("HouseholdSpend")]

demo_drop_vars = sorted(set(df_matches_demo["Variable"]))
household_drop_vars = sorted(set(df_matches_household["Variable"]))

print("DEMOSTATS variables to drop:\n", demo_drop_vars)
print("HOUSEHOLD variables to drop:\n", household_drop_vars)


DEMOSTATS variables to drop:
 ['ECYINDFINA']
HOUSEHOLD variables to drop:
 ['HSEP001', 'HSEP001S', 'HSEP002', 'HSEP003', 'HSEP004', 'HSEP005', 'HSEP006', 'HSEP007', 'HSEP008', 'HSEP009', 'HSHC022', 'HSHC025', 'HSRV011', 'HSSH006', 'HSSH015', 'HSSH019', 'HSSH044', 'HSTR032']


In [8]:
# 6) Load the original numeric data for DemoStats & HouseholdSpend
df_demo_orig = pd.read_csv("Coursework Data/DemoStats.csv")       # Replace path
df_household_orig = pd.read_csv("Coursework Data/HouseholdSpend.csv")

# 7) Drop from each DataFrame the product-specific "drop" columns
df_demo_clean = df_demo_orig.drop(columns=demo_all_drops, errors="ignore")
df_household_clean = df_household_orig.drop(columns=household_all_drops, errors="ignore")


In [10]:
# 8) Merge on ["GEO","CODE"]
df_merged = pd.merge(
    df_demo_clean,
    df_household_clean,
    on=["GEO","CODE"],
    how="inner"
)

In [11]:
# 9) Remove duplicates
df_merged.drop_duplicates(inplace=True)

In [12]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [14]:
df_merged.drop(columns=columns_to_drop, errors="ignore", inplace=True)
df_merged.drop_duplicates(inplace=True)

print("Full merged shape before sampling:", df_merged.shape)

# ------------------------------------------------------------------
# 4) SAMPLE THE DATAFRAME (e.g., 100k rows)
# ------------------------------------------------------------------
df_sample = df_merged.sample(n=100000, random_state=42)
print("Sample shape:", df_sample.shape)

NameError: name 'columns_to_drop' is not defined

In [ ]:
df_numeric = df_sample.select_dtypes(include=[np.number])
df_non_numeric = df_sample.select_dtypes(exclude=[np.number])

In [12]:
# Check for inf values
import numpy as np
mask_inf = df_numeric.isin([np.inf, -np.inf]).any(axis=1)
if mask_inf.sum() > 0:
    print("Found inf in these rows:", mask_inf.sum())

In [ ]:
imputer = SimpleImputer(strategy="median")
df_numeric_imputed = pd.DataFrame(
    imputer.fit_transform(df_numeric),
    columns=df_numeric.columns,
    index=df_numeric.index
)

In [ ]:
# 11) IQR Filter for outliers
def iqr_filter(df, multiplier=1.5):
    df_filtered = df.copy()
    for col in df_filtered.columns:
        if pd.api.types.is_numeric_dtype(df_filtered[col]):
            Q1 = df_filtered[col].quantile(0.25)
            Q3 = df_filtered[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - multiplier * IQR
            upper_bound = Q3 + multiplier * IQR
            df_filtered = df_filtered[
                (df_filtered[col] >= lower_bound) & (df_filtered[col] <= upper_bound)
            ]
    return df_filtered

df_iqr = iqr_filter(df_imputed)